# Applied Statistical Learning - Prédiction de la durée de mise en chantier en France
Dans le cadre de ce projet, nous avons exploité les données du fichier SITADEL, qui recense de manière exhaustive l'ensemble des permis de construire en France. Pour avoir un échantillon homogène, nous nous sommes concentrés sur la période 2015-2019 (i.e. avant les délais liés au Covid). Nous avons enrichi notre fichier à l'aide de deux sources externes: le fichier complet de l'Insee, agrégeant des données en open data pour l'ensemble des communes, ainsi que la grille de densité des communes de l'Insee. 
Pour faciliter la réplication de nos résultats, nous avons déposé nos données brutes sur une dropbox (1.5Go) : https://www.dropbox.com/scl/fo/b3dw5eht79785mrg7uueq/APQ7m21mQcsZzh7Uy7ASj6k?rlkey=rutboi86sxni30nqg6lzsycqh&st=n60gni1d&dl=0. 

# 1. Mise en place des chemins
Pour faire tourner l'ensemble du code de ce notebook, il est nécessaire de placer l'ensemble des fichiers de la dropbox dans un dossier data, puis d'adapter le chemin ci-dessous. 

In [ ]:
from pathlib import Path

# CHEMIN A ADAPTER
chemin_data = Path("data")   # mettre le chemin local vers le dossier data avec les fichiers de la dropbox

# Suffixes fixes
chemin_autorisation = chemin_data / "Liste-des-autorisations-durbanisme-creant-des-logements.2025-10.csv"
chemin_grilles = chemin_data / "grille_densite_7_niveaux_2019.xlsx"
chemin_dossier = chemin_data / "dossier_complet.csv"

# installer pyarrow pour lire les fichiers parquet, et openpyxl pour lire les fichiers excel
%pip install pyarrow fastparquet
%pip install openpyxl

# 2. Chargement des données
Ce premier chunk permet de sélectionner nos variables d'intérêt, et d'observer notre base avant tout filtre. Le fichier contient initialement 1,8 million de permis de construire, couvrant souvent plusieurs logements. Plusieurs variables d'intérêt sont possibles : la durée avant l'obtention de l'autorisation, la durée entre l'obtention de l'autorisation et la mise en chantier, ou la durée du chantier. Afin d'éviter de nous restreindre à des chantiers terminés, nous étudierons la **durée de mise en chantier**. Après le retrait de durées absentes ou négatives, nous avons 1,7 million de permis.

In [ ]:
# Libraries
import pandas as pd
import numpy as np

# 1. Chargement de Sitadel
# Chemin : défini en amont, vérifier que le chunk a bien été executé. 

# 1.1 Exclusion ex ante des colonnes non pertinentes
all_cols = pd.read_csv(chemin_autorisation, sep=";", skiprows=1, nrows=1).columns.tolist()

vars_mai2022 = [ #On retire les colonnes qui n'existent qu'à partir de mai 2022 (car on veut un dataset homogène dans le temps)
    "AN_DEPOT",  # "DPC_PREM", (theoriquement il faudrait la retirer, mais bon)
    "NATURE_PROJET_COMPLETEE",
    "DESTINATION_PRINCIPALE",
    "TYPE_PRINCIP_LOGTS_CREES",
    "TYPE_TRANSFO_PRINCIPAL",
    "TYPE_PRINCIP_LOCAUX_TRANSFORMES",
    "I_PISCINE",
    "I_GARAGE",
    "I_VERANDA",
    "I_ABRI_JARDIN",
    "I_AUTRE_ANNEXE",
    "RES_PERS_AGEES",
    "RES_ETUDIANTS",
    "RES_TOURISME",
    "RES_HOTEL_SOCIALE",
    "RES_SOCIALE",
    "RES_HANDICAPES",
    "RES_AUTRE",
    "NB_LGT_INDIV_PURS",
    "NB_LGT_INDIV_GROUPES",
    "NB_LGT_RES",
    "NB_LGT_COL_HORS_RES",
    "SUgbr_HEB_TRANSFORMEE",
    "SUgbr_BUR_TRANSFORMEE",
    "SUgbr_COM_TRANSFORMEE",
    "SUgbr_ART_TRANSFORMEE",
    "SUgbr_IND_TRANSFORMEE",
    "SUgbr_AGR_TRANSFORMEE",
    "SURF_ENT_TRANSFORMEE",
    "SURF_PUB_TRANSFORMEE",
]
vars_non_pertinentes = [ # on retire les variables sans pouvoir prédictif (ex : SIREN du demandeur)
    "Num_DAU",
    "SIREN_DEM",
    "SIRET_DEM",
    "DENOM_DEM",
    "CODPOST_DEM",
    "LOCALITE_DEM",
    "ADR_NUM_TER",
    "ADR_TYPEVOIE_TER",
    "ADR_LIBVOIE_TER",
    "ADR_LIEUDIT_TER",
    "ADR_LOCALITE_TER",
    "ADR_CODPOST_TER",
    "SEC_CADASTRE1",
    "NUM_CADASTRE1",
    "SEC_CADASTRE2",
    "NUM_CADASTRE2",
    "SEC_CADASTRE3",
    "NUM_CADASTRE3",
]

cols_to_drop = set(vars_mai2022 + vars_non_pertinentes)
use_cols = [c for c in all_cols if c not in cols_to_drop]

# 1.2 Chargement avec les colonnes filtrées
df = pd.read_csv(
    chemin_autorisation,
    sep=";",
    encoding="utf-8",
    skiprows=1,
    usecols=use_cols
)

# 3. Etudions nos dates
date_cols = [
    "DATE_REELLE_AUTORISATION",
    "DATE_REELLE_DOC",
    "DPC_AUT",
    "DATE_REELLE_DAACT",
    "DPC_PREM",
]

for col in date_cols:
    if col in df.columns:
        print(f"\n{col}:")
        print(f"  Type: {df[col].dtype}")
        print("  Sample values:")
        print(df[col].head(10).tolist())
        print(f"  Null count: {df[col].isna().sum()}")
    else:
        print(f"\n{col}: NOT FOUND in dataframe")


## Puis, nettoyons le dataset
def nettoyer_dataset(df: pd.DataFrame):
    df = df.copy()

    # Conversion des dates en datetime
    # 3 dates en format DD/MM/YYYY
    dmY_cols = ["DATE_REELLE_AUTORISATION", "DATE_REELLE_DOC", "DATE_REELLE_DAACT"]
    for col in dmY_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(
                df[col], errors="coerce", dayfirst=True
            )

    # 2 dates en format YYYY-MM
    for col in ["DPC_AUT", "DPC_PREM"]:
        if col in df.columns:
            # on impose un format
            df[col] = pd.to_datetime(
                df[col].astype(str).str.strip(), errors="coerce", format="%Y-%m"
            )

    # On retire les dates fausses
    min_date = pd.Timestamp("1900-01-01") 
    max_date = pd.Timestamp("2025-12-31")
    for col in dmY_cols + ["DPC_AUT", "DPC_PREM"]:
        if col in df.columns:
            mask = (df[col] < min_date) | (df[col] > max_date)
            df.loc[mask, col] = pd.NaT

    # On construit trois variables cibles, même si in fine on n'utilisera que delai_ouverture_chantier
    # Nous avons laissé notre code car nous avions essayé avec d'autres variables cibles, finalement moins faciles à prédire.
    if "DATE_REELLE_AUTORISATION" in df.columns and "DATE_REELLE_DOC" in df.columns:
        mask = df["DATE_REELLE_AUTORISATION"].notna() & df["DATE_REELLE_DOC"].notna()
        df.loc[mask, "delai_ouverture_chantier"] = (
            df.loc[mask, "DATE_REELLE_DOC"] - df.loc[mask, "DATE_REELLE_AUTORISATION"]
        ).dt.days

    if "DATE_REELLE_DAACT" in df.columns and "DATE_REELLE_DOC" in df.columns:
        mask = df["DATE_REELLE_DAACT"].notna() & df["DATE_REELLE_DOC"].notna()
        df.loc[mask, "duree_travaux"] = (
            df.loc[mask, "DATE_REELLE_DAACT"] - df.loc[mask, "DATE_REELLE_DOC"]
        ).dt.days

    if "DPC_AUT" in df.columns and "DPC_PREM" in df.columns:
        mask = df["DPC_PREM"].notna() & df["DPC_AUT"].notna()
        df.loc[mask, "duree_obtiention_autorisation"] = (
            df.loc[mask, "DPC_AUT"] - df.loc[mask, "DPC_PREM"]
        ).dt.days

    # On traite les variables cibles: conversion en numérique et suppression des valeurs négatives
    duration_cols = [
        "delai_ouverture_chantier",
        "duree_travaux",
        "duree_obtiention_autorisation",
    ]
    for col in duration_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
            df.loc[df[col] <= 0, col] = pd.NA

    # Suppression des lignes sans cibles valides (NA, null, zero or negative)
    existing_duration_cols = [c for c in duration_cols if c in df.columns]
    df = df.dropna(subset=existing_duration_cols, how="all")
    
    # On crée mois et année
    df["annee_autorisation"] = df["DATE_REELLE_AUTORISATION"].dt.year
    df["mois_autorisation"] = df["DATE_REELLE_AUTORISATION"].dt.month
    return df


df_clean = nettoyer_dataset(df)

# On règle également les codes (dep, commune, région)
colonnes_codes = [
    "DEP_CODE",
    "COMM",
    "CODGEO",
    "REG_CODE"
]

for col in colonnes_codes:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype("string")

# On enregistre en parquet pour conserver les formats
df_clean.to_parquet(
    chemin_data / "autorisations.parquet",
    engine="pyarrow",
    index=False
)

## 4. Comparaison
# Avant nettoyage
print("Df shape avant:", df.shape)
print("Nombre de lignes dans le df original:", len(df))

# Après nettoyage
print("\n Après nettoyages :")
print(
    "Lignes avec delai_ouverture_chantier non-NA:",
    df_clean["delai_ouverture_chantier"].notna().sum(),
)
print("Lignes avec duree_travaux non-NA:", df_clean["duree_travaux"].notna().sum())
print(
    "Lignes avec duree_obtiention_autorisation non-NA:",
    df_clean["duree_obtiention_autorisation"].notna().sum(),
)
print(
    "Lignes avec duree_obtiention_autorisation non-0:",
    (
        df_clean["duree_obtiention_autorisation"].notna()
        & (df_clean["duree_obtiention_autorisation"] != 0)
    ).sum(),
)
print("\nShape du df nettoyé :", df_clean.shape)
print("Nombre de lignes dans le df nettoyé :", len(df_clean))

# 3. Filtre et enrichissement des données
Avec SITADEL, nous avons des données relatives au projet de construction (caractéristiques du demandeur, caractéristiques du projet), mais nous n'avons aucune information sur l'environnement territorial, à l'exception du code commune-département-région. Par appariement à l'aide du code commune, nous ajoutons donc la grille de densité qui nous renseigne sur le type d'occupation du sol (centre urbanisé, rural dense, rural peu dense, ...) et la population communale. Nous ajoutons également des données socio-démographiques du dossier complet : taux de pauvreté communal, part de ménages imposés, nombre de résidences secondaires, etc. 

In [ ]:
# Libraries
import pandas as pd
import openpyxl

# 1. Chargement des autorisations nettoyées
# Chemin a déjà été défini plus haut

df = pd.read_parquet(chemin_data / "autorisations.parquet")

df.info()
print(df.head())

# 2. Chargement de données externes
# 2.1. Grille de densité à 7 niveaux
grille_densite = pd.read_excel(
    chemin_grilles,
    skiprows=4
)
print(grille_densite.head())
grille_densite.info()
print(grille_densite.columns.tolist())

cols_grille = [
    "CODGEO",
    "DENS",# De 1 à 7
    "PMUN17" #Pop° municipale 2017
]
grille_reduite = grille_densite[cols_grille]

# On corrige les formats
df["COMM"] = df["COMM"].astype(str).str.zfill(5)
grille_densite["CODGEO"] = grille_densite["CODGEO"].astype(str).str.zfill(5)

# On fait un left-join (i.e. on enrichit les autorisations)
df = df.merge(
    grille_reduite,
    how="left",
    left_on="COMM",
    right_on="CODGEO",
    validate="m:1"
)
# On evalue les manquants
missing_rate = df["DENS"].isna().mean()
print(f"Pourcentage sans densité: {missing_rate:.2%}")

# 2.2. Dossier complet INSEE
all_cols = pd.read_csv(chemin_dossier, sep=";", skiprows=0, nrows=1).columns.tolist()
print(all_cols)

cols_dossier_complet = [
    "CODGEO", #code Insee
    "TP6021",# taux pauvreté 60% en 2021 (retirée car trop de NA - secret statistique)
    "MED21", #Médiane des revenus fiscaux en 2021"
    "PIMP21", #part de ménages fiscaux imposés (retirée car trop de NA - secret statistique)
    "PPEN21", #Part des pensions dans le revenu fiscal (proxy pour concentration personnes âgées) (retirée car trop de NA - secret statistique)
    "DECE1621", #nbr de deces entre 2016 et 2021
    "P16_LOG", #nbr de logements dans la commune en 2022
    "P16_RP", #RP en 2016
    "P16_RSECOCC", #nbr de résidences secondaires et logements occasionnels en 2016
    "P16_LOGVAC", #nbr de logements vacants en 2016 
    "P16_MAISON", # Maisons individuelles en 2016
    "P16_APPART",  #Appartements en 2016 
    "P16_NSCOL15P", #Pop 15 ans ou plus non scolarisée en 2016
    "P16_ACTOCC15P", #Actifs occupés 15 ans ou plus en 2016
    "P16_CHOM1564", #Chômeurs 15-64 ans en 2016 (princ);
]

dossier_complet = pd.read_csv(
    chemin_dossier,
    sep=";",
    encoding="utf-8",
    skiprows=0,
    usecols=cols_dossier_complet
)

dossier_complet["CODGEO"] = dossier_complet["CODGEO"].astype(str).str.zfill(5)

# Certaines colonnes devraient être numériques mais ne le sont pas à cause des infos de secret statistique
cols_num = ["MED21"] #on ne met plus le taux de pauvreté, car trop de NA

for col in cols_num:
    dossier_complet[col] = (
        dossier_complet[col]
        .astype(str)
        .str.strip()
        .replace(INSEE_secret_stat, None)
        .str.replace("\xa0", "", regex=False)  # espace insécable
        .str.replace(" ", "", regex=False)
        .str.replace(",", ".", regex=False)
        .astype(float)
    )

# On fait un left-join (i.e. on enrichit les autorisations)
df = df.merge(
    dossier_complet,
    how="left",
    left_on="COMM",
    right_on="CODGEO",
    validate="m:1"
)

df.info()
df.to_parquet(
    chemin_data / "autorisations_enrichies.parquet",
    engine="pyarrow",
    index=False
)

# 3. Compte des valeurs manquantes pour l'ensemble des variables du dossier complet ajoutées
variable_dict = {
    # Identifiers
    "COMM": "Code INSEE de la commune (autorisations)",
    "CODGEO": "Code INSEE de la commune (sources externes)",

    # Grille densité
    "DENS": "Niveau de densité communale (1 = très dense, 7 = très peu dense)",
    "PMUN17": "Population municipale 2017",

    # Socio-éco (INSEE – dossier complet)
    "TP6021": "Taux de pauvreté à 60% du niveau de vie médian (2021)",
    "MED21": "Médiane des revenus fiscaux (€) en 2021",
    "PIMP21": "Part des ménages fiscaux imposés (%)",
    "PPEN21": "Part des pensions dans le revenu fiscal (%)",
    "DECE1621": "Nombre de décès cumulés entre 2016 et 2021",
    "P16_LOG": "Nombre total de logements (2022)",
    "P16_RP": "Nombre de résidences principales (2016)",
    "P16_RSECOCC": "Résidences secondaires et logements occasionnels (2016)",
    "P16_LOGVAC": "Logements vacants (2016)",
    "P16_MAISON": "Maisons individuelles (2016)",
    "P16_APPART": "Appartements (2016)",
    "P16_NSCOL15P": "Population 15+ ans non scolarisée (2016)",
    "P16_ACTOCC15P": "Actifs occupés 15+ ans (2016)",
    "P16_CHOM1564": "Chômeurs 15–64 ans (2016)"
}

vars_added = [v for v in variable_dict.keys() if v in df.columns]

summary_table = (
    pd.DataFrame({
        "variable": vars_added,
        "description": [variable_dict[v] for v in vars_added],
        "share_na": [df[v].isna().mean() for v in vars_added]
    })
    .sort_values("share_na", ascending=False)
    .reset_index(drop=True)
)

print(summary_table)


# 4. Prepocessing
Le pré-traitement vise à construire un échantillon homogène, économiquement interprétable et compatible avec nos estimations, en premier lieu un lasso. À partir des autorisations enrichies, les observations sans délai d’ouverture de chantier sont exclues, tout comme les départements d’outre-mer afin de limiter l’hétérogénéité. L’analyse est restreinte à la période 2015–2019, afin de réduire les temps de calcul, et ne pas devoir prendre en compte le bousculement majeur qu'a été le premier confinement. Les annulations sont retirées et les délais sont élagués en fixant un plafond à deux ans (730 jours), entre le 95e et 99e percentile, afin de réduire l'influence des outliers. 

Enfin, les variables numériques sont standardisées afin de rendre les coefficients comparables sous pénalisation ℓ₁, tandis que les variables qualitatives sont encodées en indicatrices via un one-hot encoding. S'inspirant des codes des TD, l’ensemble du pré-traitement est intégré dans un pipeline sklearn. Nous conservons à la fin de ce prétraitement 530 000 observations. 

In [ ]:
import numpy as np
import pandas as pd

# LIBRAIRIES de sklearn
# Pour le preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# A) Préparation des données pour le LASSO

# Variables supplémentaires à retirer du jeu de données final
vars_inutiles_delai_ouverture = [
    "COMM",
    "REG_CODE",
    "REG_LIBELLE",
    "DEP_LIBELLE",
    "NUM_DAU",
    "APE_DEM",
    "CJ_DEM",
    "duree_obtiention_autorisation",
    "DATE_REELLE_AUTORISATION",
    "DATE_REELLE_DAACT",
    "DATE_REELLE_DOC",
    "DPC_AUT",
    "DPC_PREM",
    "DPC_DOC",
    "DPC_DERN", 
    "duree_travaux"]

# Variables à traiter en One-Hot Encoding
vars_categ = [
    "DEP_CODE", 
    "TYPE_DAU", 
    "ETAT_DAU", 
    "CAT_DEM",
    "ZONE_OP",
    "NATURE_PROJET_DECLAREE",
    "UTILISATION",
    "RES_PRINCIP_OU_SECOND",
    "TYP_ANNEXE",
    "RESIDENCE" ] #on ne met pas la grille de densité pour ne pas perdre la nature hiérarchique de la variable (1 à 7)

df = pd.read_parquet(chemin_data / "autorisations_enrichies.parquet")

# Filtrage des régions et des années
regions_outremer = [
    "Guadeloupe", "Martinique", "Guyane",
    "La Réunion", "Mayotte"
]

df_filtre_delai_ouverture = df.dropna(subset=["delai_ouverture_chantier"])
df_filtre_delai_ouverture = df_filtre_delai_ouverture[
    ~df_filtre_delai_ouverture["REG_LIBELLE"].isin(regions_outremer)
]

# On ne conserve que 2015-2019, et on retire les outliers (annulations, et délais dépassent 2 ans, entre 95 et 99e percentile)
p95 = df_filtre_delai_ouverture["delai_ouverture_chantier"].quantile(0.95)
p99 = df_filtre_delai_ouverture["delai_ouverture_chantier"].quantile(0.99)
print(f"Seuil 95e percentile : {p95:.1f} jours")
print(f"Seuil 99e percentile : {p99:.1f} jours")

df_filtre_delai_ouverture = df_filtre_delai_ouverture[(df_filtre_delai_ouverture["annee_autorisation"] >= 2015) & (df_filtre_delai_ouverture["annee_autorisation"] <= 2019)]
df_filtre_delai_ouverture = df_filtre_delai_ouverture[df_filtre_delai_ouverture["delai_ouverture_chantier"] <= 730]
df_filtre_delai_ouverture = df_filtre_delai_ouverture[df_filtre_delai_ouverture["ETAT_DAU"] != 4] #on retire les annulations

# Nettoyage et conversion des types sur l'échantillon
df_filtre_delai_ouverture = df_filtre_delai_ouverture.drop(columns=vars_inutiles_delai_ouverture, errors="ignore")

# Suppression des lignes sans variable cible
df_model = df_filtre_delai_ouverture.dropna(subset=["delai_ouverture_chantier"])

# On définit X et y
y = df_model["delai_ouverture_chantier"]
X = df_model.drop(columns=["delai_ouverture_chantier"])

# Colonnes numériques
num_cols = X.select_dtypes(
    include=["float", "int", "bool"]
).columns.tolist()

# Colonnes catégorielles
cat_cols = X.select_dtypes(
    include=["string"]
).columns.tolist()

# On gère les NAs des variables explicatives
cols_utiles = num_cols + cat_cols
n_before_na = len(X)

# Suppression des lignes avec NA sur les variables explicatives
X = X.dropna(subset=cols_utiles)
y = y.loc[X.index] 

n_after_na = len(X)
drop_rate = 100 * (1 - n_after_na / n_before_na)

print(f"Observations après filtrage NA : {n_after_na:,}")

# B) Pipeline de pré-traitement et modèle LASSO
preprocess = ColumnTransformer(
    transformers=[
        # Mise à l'échelle des variables numériques (StandardScaler)
        ("num", StandardScaler(), num_cols), 
        # Encodage One-Hot des variables catégorielles
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ],
    remainder='drop' # On jette ce qui reste
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)


# 5. Régression LASSO
Notre première estimation repose sur un modèle LASSO. La pénalisation ℓ₁, paramétrée ici par un coefficient de régularisation fixé à (\alpha = 0{,}1) permet de limiter le sur-apprentissage et d’opérer une sélection automatique des variables en contraignant certains coefficients à zéro. Le modèle est entraîné sur l’échantillon d’apprentissage, puis évalué séparément sur les données de test à l’aide de métriques standards de régression (RMSE, MAE et (R^2)), afin de comparer les performances in-sample et out-of-sample. L’analyse est complétée par l’extraction des coefficients non nuls, commentés dans le rapport.

Enfin, une spécification alternative est testée en modélisant le logarithme du délai, dans le but de réduire l’asymétrie de la variable cible (i.e. une queue de distribution épaisse) : celle-ci requiert une adaptation du paramètre alpha. Nous avons utilisé LassoCV, proche de GridSearchCV présenté en TD mais plus rapide pour notre cadre d'analyse, afin de trouver l'alpha optimal, plus petit que celui en niveau afin de ne pas augmenter la pénalisation. Les prédictions obtenues n'améliorent néanmoins pas notre estimation : la variance prédite semble resserée par la transformation. 


In [ ]:
# Models
from sklearn.linear_model import Lasso

# Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

lasso = Lasso(alpha=0.1, max_iter=10000)

lasso_pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", lasso)
])

lasso_pipeline.fit(X_train, y_train)

y_train_pred = lasso_pipeline.predict(X_train)
y_test_pred = lasso_pipeline.predict(X_test)

test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"RMSE train : {train_rmse:.2f}")
print(f"RMSE test  : {test_rmse:.2f}")
print(f"MAE train  : {train_mae:.2f}")
print(f"MAE test   : {test_mae:.2f}")
print(f"R² train   : {train_r2:.3f}")
print(f"R² test    : {test_r2:.3f}")

print('R2 sur le test: {}'.format(lasso_pipeline.score(X_test, y_test)))


# Nombre de coefficients non nuls
coefs = lasso_pipeline.named_steps["model"].coef_
nb_nonzero = np.sum(coefs != 0)
print("Nombre de coefficients non nuls :", nb_nonzero)

# Liste des coefficients non nuls 
feature_names = (
    list(num_cols) +
    list(lasso_pipeline.named_steps["preprocess"]
         .named_transformers_["cat"]
         .get_feature_names_out(cat_cols))
)
coef_df = (
    pd.DataFrame({"variable": feature_names, "coef": coefs})
      .assign(abs_coef=lambda d: d.coef.abs())
)
print("\nVariables les plus explicatives :")
with pd.option_context("display.max_rows", None):
    print(coef_df.query("coef != 0").sort_values("abs_coef", ascending=False))
print(coef_df.query("coef != 0").sort_values("abs_coef", ascending=False))

# Variables non explicatives
print("\nVariables éliminées par le LASSO :")
print(coef_df.query("coef == 0").variable.tolist())


#### Version avec transformation log de y ####
y_log = np.log1p(y)

X_train2, X_test2, y_train2, y_test2 = train_test_split(
    X, y_log,
    test_size=0.2,
    random_state=36
)

# On doit modifier un peu notre pipeline, notamment pour l'hyperparamètre alpha: voir script plus bas
lasso_log = Lasso(alpha=0.001, max_iter=10_000)

lasso_log_pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", lasso_log)
])

lasso_log_pipeline.fit(X_train2, y_train2)

# On calcule les prédictions du log
y_train_log_pred = lasso_log_pipeline.predict(X_train2)
y_test_log_pred = lasso_log_pipeline.predict(X_test2)

# On revient à l'échelle originale pour la comparabilité
y_train_pred = np.expm1(y_train_log_pred)
y_test_pred = np.expm1(y_test_log_pred)

y_train_true = np.expm1(y_train2)
y_test_true = np.expm1(y_test2)

# On calcule les métriques
train_rmse = np.sqrt(mean_squared_error(y_train_true, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test_true, y_test_pred))

train_mae = mean_absolute_error(y_train_true, y_train_pred)
test_mae = mean_absolute_error(y_test_true, y_test_pred)

train_r2 = r2_score(y_train_true, y_train_pred)
test_r2 = r2_score(y_test_true, y_test_pred)

print("\n[LASSO – log(durée + 1), évalué sur l’échelle originale]")
print(f"RMSE train : {train_rmse:.2f}")
print(f"RMSE test  : {test_rmse:.2f}")
print(f"MAE train  : {train_mae:.2f}")
print(f"MAE test   : {test_mae:.2f}")
print(f"R² train   : {train_r2:.3f}")
print(f"R² test    : {test_r2:.3f}")

In [ ]:
# Choix de l'hyperparamètre alpha 
from sklearn.linear_model import LassoCV

lasso_cv = LassoCV(
    alphas=np.logspace(-3, 0, 10),
    cv=5,
    max_iter=10_000,
    n_jobs=-1
)

lasso_log_pipeline = Pipeline([
    ("preprocess", preprocess),
    ("model", lasso_cv)
])

lasso_log_pipeline.fit(X_train2, y_train2)

print("Alpha optimal :", lasso_cv.alpha_)


# 6. Random forest

Nous conservons la même plage d’étude (2015–2019) et cherchons à améliorer le pouvoir prédictif du modèle en recourant à une méthode d’ensemble non paramétrique, la forêt aléatoire. Cela devrait nous permettre de capturer des relations non linéaires et des interactions complexes entre variables explicatives, difficilement prises en compte par les modèles linéaires.

Le premier chunk nous permet de choisir les hyperparamètres du modèle à l’aide d’une procédure de recherche aléatoire avec élimination successive (*HalvingRandomSearchCV*). Cette méthode évalue un grand nombre de configurations sur un nombre limité de ressources, puis concentre progressivement l’effort de calcul sur les configurations les plus prometteuses. La ressource considérée ici est le nombre d’arbres de la forêt (*n_estimators*), augmenté itérativement de 50 à 350.

La recherche porte sur trois hyperparamètres clés :
- la profondeur maximale des arbres (*max_depth*), qui contrôle la complexité du modèle ;
- le nombre minimal d’observations par feuille (*min_samples_leaf*), qui joue un rôle de régularisation ;
- la proportion de variables considérées à chaque séparation (*max_features*), qui accroît la diversité des arbres.

La performance des modèles est évaluée par validation croisée à trois plis, en utilisant le coefficient de détermination \(R^2\) comme fonction de score, conformément au cadre de régression étudié. Les hyperparamètres retenus correspondent à la configuration maximisant la performance moyenne en validation croisée.



In [ ]:
# On va chercher à tuner automatiquement les hyperparamètres du modèle

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint

param_dist = {
    "model__max_depth": [6, 8, 10, 12],
    "model__min_samples_leaf": [5, 10, 20, 50],
    "model__max_features": ["sqrt", 0.3, 0.5],
}

rf_pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", RandomForestRegressor(
        random_state=42,
        n_jobs=-1
    ))
])

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

search = HalvingRandomSearchCV(
    rf_pipeline,
    param_distributions=param_dist,
    resource="model__n_estimators",
    min_resources=50,   
    max_resources = 350,
    factor=3,
    scoring="r2",
    cv=3,
    random_state=36,
    verbose=2,
    n_jobs=1
)

search.fit(X_train, y_train)

print("Best parameters:")
print(search.best_params_)

print("Best CV R²:")
print(search.best_score_)

#Best parameters: {'model__min_samples_leaf': 10, 'model__max_features': 0.5, 'model__max_depth': 12, 'model__n_estimators': 150}
# Best CV R²: 0.16358587685634499


In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", RandomForestRegressor(
        max_depth=12, # Avons modifié la profondeur maximale
        #min_samples_split=10,# à réduire, imho
        min_samples_leaf=10, # également essayer de réduire
        max_features=0.5,# uniquement sur notre dernière itération
        n_estimators=150,
        n_jobs=-1,
        random_state=36
    ))
]) 

rf_pipeline.fit(X_train, y_train)

y_train_pred_rf = rf_pipeline.predict(X_train)
y_test_pred_rf  = rf_pipeline.predict(X_test)

train_rmse_rf = np.sqrt(mean_squared_error(y_train, y_train_pred_rf))
test_rmse_rf  = np.sqrt(mean_squared_error(y_test, y_test_pred_rf))

train_mae_rf = mean_absolute_error(y_train, y_train_pred_rf)
test_mae_rf  = mean_absolute_error(y_test, y_test_pred_rf)

train_r2_rf = r2_score(y_train, y_train_pred_rf)
test_r2_rf  = r2_score(y_test, y_test_pred_rf)

print(f"RMSE train : {train_rmse_rf:.2f}")
print(f"RMSE test  : {test_rmse_rf:.2f}")
print(f"MAE train  : {train_mae_rf:.2f}")
print(f"MAE test   : {test_mae_rf:.2f}")
print(f"R² train   : {train_r2_rf:.3f}")
print(f"R² test    : {test_r2_rf:.3f}")

print('Classification accuracy on test is: {}'.format(rf_pipeline.score(X_test, y_test)))

# 7. Gradient boosting
En application du cours d'Applied Statistical Learning, nous mobilisons une méthode de gradient boosting, qui repose sur une minimisation itérative du risque empirique par descente de gradient, chaque itération ajoutant un arbre faiblement profond afin de corriger les erreurs résiduelles du modèle précédent.
Si le cours s’est principalement appuyé sur AdaBoost et XGBoost, nous retenons ici LightGBM, qui implémente le même principe théorique tout en introduisant des optimisations algorithmiques particulièrement adaptées aux grands échantillons et aux espaces de variables de dimension élevée.
Compte tenu de la taille de notre base (>500 000 observations, et une centaine de features après prétraitement), LightGBM présente un coût computationnel nettement plus acceptable sur nos machines personnelles.


https://blent.ai/blog/a/lightgbm-mieux-que-xgboost



In [ ]:
import numpy as np
import pandas as pd

import lightgbm as lgb
from lightgbm import LGBMRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# On reprend le dataset nettoyé et enrichi
y = df_model["delai_ouverture_chantier"]
X = df_model.drop(columns=["delai_ouverture_chantier"])

# On retire les colonnes inutiles pour LGBM (identifiants et variables avec trop de NA, en object, ignorés par scikit mais pas lightgbm)
cols_to_drop_lgbm = [
    "CODGEO_x",
    "CODGEO_y",
    "TP6021",
    "PIMP21",
    "PPEN21",
]
X = X.drop(columns=[c for c in cols_to_drop_lgbm if c in X.columns])

cat_cols = X.select_dtypes(include=["string", "object"]).columns.tolist()
for c in cat_cols:
    X[c] = X[c].astype("category")

num_cols = X.select_dtypes(include=["float", "int", "bool"]).columns.tolist()

# On crée des jeux d'entraînement  et test
X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.20, random_state=36)
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.50, random_state=36)

# Passons au modèle : regression L1 ou poisson; 
model = LGBMRegressor(
    boosting_type="gbdt",
    objective="regression_l1",
    n_estimators=5000,
    learning_rate=0.03,
    num_leaves=127,
    min_child_samples=50,
    subsample=0.8,
    subsample_freq=1,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1
)

# Early stopping = callback + eval_set obligatoire. :contentReference[oaicite:3]{index=3}
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric="mae",
    callbacks=[
        lgb.early_stopping(stopping_rounds=200),
        lgb.log_evaluation(period=50)
    ],
)

# best_iteration_ est renseigné si early_stopping() est utilisé. :contentReference[oaicite:4]{index=4}
y_pred = model.predict(X_test, num_iteration=model.best_iteration_)

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"MAE  : {mae:.2f} jours")
print(f"RMSE : {rmse:.2f} jours")
print(f"R²   : {r2:.3f}")

# Imprimons l'importance des variables
imp = pd.Series(model.feature_importances_, index=model.feature_name_).sort_values(ascending=False)
print(imp.head(30))

In [ ]:
%pip install -q lightgbm
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# On réutilise le même preprocess que précédemment
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.2,
    random_state=36
)

lgbm = LGBMRegressor(
    objective="regression",
    n_estimators=5000,            # assez grand → early stopping
    learning_rate=0.03,           # plus lent mais plus stable
    num_leaves=63,                # complexité contrôlée (≈ arbre depth 7–8)
    max_depth=-1,                 # laisse num_leaves gouverner
    min_child_samples=100,        # régularisation adaptée gros N
    subsample=0.8,
    subsample_freq=1,
    colsample_bytree=0.7,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1
)

lgbm_pipeline = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("model", lgbm)
    ]
)

lgbm_pipeline.fit(
    X_tr,
    y_tr,
    model__eval_set=[(X_val, y_val)],
    model__eval_metric="rmse",
    model__callbacks=[
        lgb.early_stopping(stopping_rounds=100),
        lgb.log_evaluation(period=50)
    ]
)

y_train_pred = lgbm_pipeline.predict(X_tr)
y_val_pred   = lgbm_pipeline.predict(X_val)
y_test_pred  = lgbm_pipeline.predict(X_test)

rmse_train = mean_squared_error(y_tr, y_train_pred, squared=False)
rmse_val   = mean_squared_error(y_val, y_val_pred, squared=False)
rmse_test  = mean_squared_error(y_test, y_test_pred, squared=False)

print(f"RMSE train : {rmse_train:.2f}")
print(f"RMSE valid : {rmse_val:.2f}")
print(f"RMSE test  : {rmse_test:.2f}")

# 8. Réseau de neurones

Dans cette section, nous mobilisons un modèle de réseau de neurones artificiels de type Multilayer Perceptron afin d’évaluer si une flexibilité accrue permet d’améliorer la prédiction des délais de mise en chantier. Nous commençons par évaluer les performances d'un premier modèles dont les hyperparamètres sont fixés de manière relativement arbitraire quoique logiques : l'idée est d'abord de ne pas complexifier outre mesure l'architecture du modèle avec de trop nombreuses couches et de lui permettre de tourner en un temps raisonnable. 

## A) Modèle principal

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp_pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", MLPRegressor(
        hidden_layer_sizes=(32, 16),   
        activation="relu",
        solver="adam",
        alpha=1e-4,
        learning_rate="adaptive",
        learning_rate_init=0.001,
        batch_size=256,               
        max_iter=300,                  
        early_stopping=True,
        n_iter_no_change=15,
        tol=1e-4,
        random_state=42
    ))
])

mlp_pipeline.fit(X_train, y_train)

# Prédictions
y_train_pred_mlp = mlp_pipeline.predict(X_train)
y_test_pred_mlp  = mlp_pipeline.predict(X_test)

# Métriques
train_rmse_mlp = np.sqrt(mean_squared_error(y_train, y_train_pred_mlp))
test_rmse_mlp  = np.sqrt(mean_squared_error(y_test, y_test_pred_mlp))

train_mae_mlp = mean_absolute_error(y_train, y_train_pred_mlp)
test_mae_mlp  = mean_absolute_error(y_test, y_test_pred_mlp)

train_r2_mlp = r2_score(y_train, y_train_pred_mlp)
test_r2_mlp  = r2_score(y_test, y_test_pred_mlp)

print(f"[MLP] RMSE train : {train_rmse_mlp:.2f}")
print(f"[MLP] RMSE test  : {test_rmse_mlp:.2f}")
print(f"[MLP] MAE train  : {train_mae_mlp:.2f}")
print(f"[MLP] MAE test   : {test_mae_mlp:.2f}")
print(f"[MLP] R² train   : {train_r2_mlp:.3f}")
print(f"[MLP] R² test    : {test_r2_mlp:.3f}")



## B) Tests de plusieurs hyperparamètres

Nous testons ici d'autres valeurs d'hyperparamètres. Les résultats sont dans le rapport car il faut savoir que le code met environ 15 minutes à tourner. L'amélioration des résultats avec ces différentes configurations est de toutes façons marginale. 

In [ ]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

hls_grid = [(32, 16), (64, 32), (64, 32, 16)]
alpha_grid = [1e-4, 1e-3]
lr_init_grid = [1e-3, 5e-4]

results = []

for hls in hls_grid:
    for alpha in alpha_grid:
        for lr_init in lr_init_grid:
            mlp_pipe = Pipeline(steps=[
                ("preprocess", preprocess),
                ("model", MLPRegressor(
                    hidden_layer_sizes=hls,
                    activation="relu",
                    solver="adam",
                    alpha=alpha,
                    learning_rate="adaptive",
                    learning_rate_init=lr_init,
                    batch_size=256,
                    max_iter=300,
                    early_stopping=True,
                    n_iter_no_change=15,
                    tol=1e-4,
                    random_state=42
                ))
            ])

            mlp_pipe.fit(X_train, y_train)

            y_train_pred = mlp_pipe.predict(X_train)
            y_test_pred  = mlp_pipe.predict(X_test)

            rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
            rmse_test  = np.sqrt(mean_squared_error(y_test, y_test_pred))
            mae_train  = mean_absolute_error(y_train, y_train_pred)
            mae_test   = mean_absolute_error(y_test, y_test_pred)
            r2_train   = r2_score(y_train, y_train_pred)
            r2_test    = r2_score(y_test, y_test_pred)

            results.append({
                "hidden_layer_sizes": hls,
                "alpha": alpha,
                "learning_rate_init": lr_init,
                "RMSE_train": rmse_train,
                "RMSE_test": rmse_test,
                "MAE_train": mae_train,
                "MAE_test": mae_test,
                "R2_train": r2_train,
                "R2_test": r2_test
            })

# Tri par RMSE test (ou MAE test)
results_sorted = sorted(results, key=lambda d: d["RMSE_test"])

for r in results_sorted[:8]:
    print(
        f"hls={r['hidden_layer_sizes']}, alpha={r['alpha']}, lr_init={r['learning_rate_init']} | "
        f"RMSE tr={r['RMSE_train']:.2f}, te={r['RMSE_test']:.2f} | "
        f"R2 tr={r['R2_train']:.3f}, te={r['R2_test']:.3f}"
    )